In [ ]:
pip install ragas==0.1.9 -q

In [65]:
import os
import pandas as pd

In [1]:
os.chdir('../../')
os.getcwd()

'/teamspace/studios/this_studio/Legal_Expert_Contract_Advisor_Using_Precision_RAG'

In [2]:
import yaml

def load_config():
    with open("data/evaluation/raptor.json", "r") as f:
        return yaml.safe_load(f)
qa = load_config()

In [3]:
qa

[{'question': 'Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?',
  'answer': 'Except in the case of fraud the Sellers have no liability for breach of representations and warranties (See section 10.01).'},
 {'question': 'Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the result of the Sellers’ gross negligence?',
  'answer': 'No.'},
 {'question': 'How much is the escrow amount?',
  'answer': 'The escrow amount is equal to $1,000,000.'},
 {'question': 'Is escrow amount greater than the Retention Amount?',
  'answer': 'No.'},
 {'question': 'What is the purpose of the escrow?',
  'answer': 'To serve as a recourse of the Buyer in case of post-closing adjustments of the purchase price. (See section 2.07(e)).'},
 {'question': 'May the Escrow Amount serve as a recourse for the Buyer in case of breach of r

In [4]:
question = [q['question'] for q in qa]
ground_truth = [q['answer'] for q in qa]

In [108]:
question

['Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?',
 'Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the result of the Sellers’ gross negligence?',
 'How much is the escrow amount?',
 'Is escrow amount greater than the Retention Amount?',
 'What is the purpose of the escrow?',
 'May the Escrow Amount serve as a recourse for the Buyer in case of breach of representations by the Company?',
 'Are there any conditions to the closing?',
 'Are Change of Control Payments considered a Seller Transaction Expense?',
 'Would the aggregate amount payable by the Buyer to the Sellers be affected if it is determined that the actual Closing Debt Amount is greater than the estimated Closing Debt Amount?',
 'Does the Buyer need to pay the Employees Closing Bonus Amount directly to the Company’s employees?',
 'Does 

In [109]:
ground_truth

['Except in the case of fraud the Sellers have no liability for breach of representations and warranties (See section 10.01).',
 'No.',
 'The escrow amount is equal to $1,000,000.',
 'No.',
 'To serve as a recourse of the Buyer in case of post-closing adjustments of the purchase price. (See section 2.07(e)).',
 'No.',
 'No as the signing and closing are simultaneous.',
 'Yes. (See defining of Sellers Transaction Expenses).',
 'Yes. (See Section 2.07).',
 'No. (See Section 2.10).',
 'No. Only the Company provides such a representation.',
 'No.',
 'If the assignment is to an Affiliate or purchaser of all of the Buyer’s assets no consent is required. Otherwise the consent of the Company and the Seller Representative is required.',
 'No. If the assignment is not part of a sale of all or substantially all of the Buyer’s assets the assignment requires the consent of the Company and the Seller’s Representative.']

In [110]:
import requests

url = "http://127.0.0.1:8000/qa/test_ragas"
answers = []
contexts = []
for q in question:
    data = {
        "query": q
    }
    response = requests.post(url, json=data)

    print(response.json()['response'])

    answers.append(response.json()['response'])
    contexts.append(response.json()['context'])

The Sellers are responsible for a breach of representations and warranties to the extent of their Pro Rata Percentage, severally and not jointly. Each Seller is individually responsible for their own breach of representations and warranties, and they are required to indemnify the Buyer for any Losses arising from such breaches. The Sellers' liability is based on their individual representations and warranties, and they are not jointly liable for each other's breaches.
No, the Sellers would not be responsible if inaccuracies in the representation provided by them are the result of their gross negligence, as the Sellers’ Representative shall not be held liable for actions or omissions unless in cases of gross negligence, bad faith, or willful misconduct. The Sellers’ Representative would be indemnified by each Seller for any Losses arising out of serving as the Sellers’ Representative, except for Losses caused by gross negligence, bad faith, or willful misconduct. The Sellers' Representa

In [111]:
contexts = [[con] for con in contexts]

In [112]:
contexts

[['Liability.  The Sellers’ Representative shall not be held liable by any of the Sellers for actions or omissions in exercising or failing to exercise all or any of the power and authority of the Sellers’ Representative pursuant to this Agreement, except in the case of the Sellers’ Representative’s gross negligence, bad faith or willful misconduct.  The Sellers’ Representative shall be entitled to rely on the advice of counsel, public accountants or other independent experts that it reasonably determines to be experienced in the matter at issue, and will not be liable to any Seller for any action taken or omitted to be taken in good faith based on such advice.  Each Seller will severally (and not jointly) indemnify (in accordance with its Pro Rata Percentage) the Sellers’ Representative from any Losses arising out of its serving as the Sellers’ Representative hereunder, except for Losses arising out of or caused by the Sellers’ Representative’s gross negligence, bad faith or willful m

In [113]:
len(answers), len(contexts), len(question), len(ground_truth)

(14, 14, 14, 14)

In [114]:
from datasets import Dataset
# To dict
data = {
    "question": question,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [115]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 14
})

In [116]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

Evaluating:   0%|          | 0/56 [00:00<?, ?it/s]

In [117]:
df = result.to_pandas()

In [118]:
df.to_csv("data/evaluation/character_evaluration_test_ragas_001_raptor.csv")

In [ ]:
df.to_csv("data/evaluation/semantic_evaluration_test_ragas_001_raptor.csv")

In [ ]:
df.to_csv("data/evaluation/recursive_evaluration_test_ragas_001_raptor.csv")

Recursive Split Scores Ragas

In [96]:
recursive_df = pd.read_csv("data/evaluation/recursive_evaluration_test_ragas_001_raptor.csv")
recursive_df

,Unnamed: 0,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,0,Under what circumstances and to what extent th...,The Sellers are responsible for a breach of re...,['Liability. The Sellers’ Representative shal...,Except in the case of fraud the Sellers have n...,1.0,0.000000,0.500000,0.977159
1,1,Would the Sellers be responsible if after the ...,"No, the Sellers would not be responsible if in...",['Liability. The Sellers’ Representative shal...,No.,1.0,1.000000,0.666667,0.991710
2,2,How much is the escrow amount?,"I'm sorry, but without additional context or i...",['Escrow.\n\nEscrow.\n\nEscrow.\n\nEscrow.'],"The escrow amount is equal to $1,000,000.",0.0,0.000000,1.000000,0.000000
3,3,Is escrow amount greater than the Retention Am...,I don't have enough information to determine i...,['Escrow.\n\nEscrow.\n\nEscrow.\n\nEscrow.'],No.,0.0,0.000000,0.000000,0.000000
4,4,What is the purpose of the escrow?,The purpose of the escrow is to hold the Escro...,"['At Closing, Buyer will deposit the Escrow Am...",To serve as a recourse of the Buyer in case of...,1.0,1.000000,0.333333,1.000000
5,5,May the Escrow Amount serve as a recourse for ...,Recovery from the Escrow Amount can be the Buy...,['Recovery from the Escrow Amount shall consti...,No.,1.0,0.000000,0.800000,0.890738
6,6,Are there any conditions to the closing?,"No, there are no conditions to the closing tha...",['No Acquired Company will be required to incl...,No as the signing and closing are simultaneous.,1.0,0.000000,0.750000,0.878652
7,7,Are Change of Control Payments considered a Se...,"Yes, Change of Control Payments are considered...",['“Seller Transaction Expenses” means all cost...,Yes. (See defining of Sellers Transaction Expe...,1.0,1.000000,0.250000,0.977536
8,8,Would the aggregate amount payable by the Buye...,"Yes, if the actual Closing Debt Amount is grea...",['“Closing Cash Amount” means the sum of all c...,Yes. (See Section 2.07).,1.0,0.333333,0.333333,0.964729
9,9,Does the Buyer need to pay the Employees Closi...,"Yes, the Buyer needs to deposit the Employee C...",['Payments of Employee Closing Bonus. Prior to...,No. (See Section 2.10).,1.0,0.000000,1.000000,0.923070


In [104]:
# calculate the average score of context precision per total score

context_precision_score = recursive_df['context_precision'].sum() / recursive_df['context_precision'].count() * 100
context_recall_score = recursive_df['context_recall'].sum() / recursive_df['context_recall'].count() * 100
faithfulness_score = recursive_df['faithfulness'].sum() / recursive_df['faithfulness'].count() * 100
answer_relevancy_score = recursive_df['answer_relevancy'].sum() / recursive_df['answer_relevancy'].count() * 100

avg_total_score = (context_precision_score + context_recall_score + faithfulness_score + answer_relevancy_score) / 4

avg_context_score = (context_precision_score + context_recall_score) / 2

avg_generate_score = (faithfulness_score + answer_relevancy_score) / 2

print("Average of All Scores is :: ", avg_total_score, " %")
print("Average of Context Scores is :: ", avg_context_score, " %")
print("The Avet]rage Generation Score :: ", avg_generate_score, " %")
print("Context Precision Score :: ", context_precision_score, " %")
print("Context Recall Score :: ", context_recall_score, " %")
print("Answer Relevancy Score :: ", answer_relevancy_score, " %")
print("Faithfulness Score :: ", faithfulness_score, " %")

Average of All Scores is ::  61.720998905160805  %
Average of Context Scores is ::  58.333333329404766  %
The Avet]rage Generation Score ::  65.10866448091684  %
Context Precision Score ::  78.57142856357143  %
Context Recall Score ::  38.0952380952381  %
Answer Relevancy Score ::  74.50304324754796  %
Faithfulness Score ::  55.714285714285715  %


Split With Semantic Chunking Ragas Score

In [105]:
semantic_df = pd.read_csv("data/evaluation/semantic_evaluration_test_ragas_001_raptor.csv")
semantic_df

,Unnamed: 0,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,0,Under what circumstances and to what extent th...,The Sellers are responsible for a breach of re...,['Liability. The Sellers’ Representative shal...,Except in the case of fraud the Sellers have n...,1.0,0.000000,0.666667,0.978324
1,1,Would the Sellers be responsible if after the ...,"No, the Sellers would not be responsible if in...",['Liability. The Sellers’ Representative shal...,No.,1.0,0.857143,0.666667,0.991239
2,2,How much is the escrow amount?,I don't have enough information to provide the...,['Escrow.\n\nEscrow.\n\nEscrow.\n\nEscrow.'],"The escrow amount is equal to $1,000,000.",0.0,0.000000,1.000000,0.000000
3,3,Is escrow amount greater than the Retention Am...,I don't have enough information to determine i...,['Escrow.\n\nEscrow.\n\nEscrow.\n\nEscrow.'],No.,0.0,0.000000,0.000000,0.000000
4,4,What is the purpose of the escrow?,The purpose of the escrow is to hold the Escro...,"['At Closing, Buyer will deposit the Escrow Am...",To serve as a recourse of the Buyer in case of...,1.0,1.000000,0.500000,1.000000
5,5,May the Escrow Amount serve as a recourse for ...,Recovery from the Escrow Amount can be the Buy...,['Recovery from the Escrow Amount shall consti...,No.,1.0,0.000000,0.800000,0.882947
6,6,Are there any conditions to the closing?,"No, there are no conditions to the closing tha...",['No Acquired Company will be required to incl...,No as the signing and closing are simultaneous.,0.0,0.000000,0.500000,0.878652
7,7,Are Change of Control Payments considered a Se...,"Yes, Change of Control Payments are considered...",['“Seller Transaction Expenses” means all cost...,Yes. (See defining of Sellers Transaction Expe...,1.0,1.000000,0.500000,1.000000
8,8,Would the aggregate amount payable by the Buye...,"Yes, if the actual Closing Debt Amount is grea...",['Purchase Price Adjustment. If any of the Cl...,Yes. (See Section 2.07).,1.0,1.000000,0.000000,0.931779
9,9,Does the Buyer need to pay the Employees Closi...,"Yes, the Buyer needs to deposit the Employee C...",['Payments of Employee Closing Bonus. Prior to...,No. (See Section 2.10).,1.0,0.000000,0.666667,0.902145


In [106]:
# calculate the average score of context precision per total score

context_score = semantic_df['context_precision'].sum() / semantic_df['context_precision'].count() * 100
context_score = semantic_df['context_recall'].sum() / semantic_df['context_recall'].count() * 100
faithfulness_score = semantic_df['faithfulness'].sum() / semantic_df['faithfulness'].count() * 100
answer_relevancy_score = semantic_df['answer_relevancy'].sum() / semantic_df['answer_relevancy'].count() * 100

avg_total_score = (context_precision_score + context_recall_score + faithfulness_score + answer_relevancy_score) / 4

avg_context_score = (context_precision_score + context_recall_score) / 2

avg_generate_score = (faithfulness_score + answer_relevancy_score) / 2

print("Average of All Scores is :: ", avg_total_score, " %")
print("Average of Context Scores is :: ", avg_context_score, " %")
print("The Avet]rage Generation Score :: ", avg_generate_score, " %")
print("Context Precision Score :: ", context_precision_score, " %")
print("Context Recall Score :: ", context_recall_score, " %")
print("Answer Relevancy Score :: ", answer_relevancy_score, " %")
print("Faithfulness Score :: ", faithfulness_score, " %")

Average of All Scores is ::  63.59638603826781  %
Average of Context Scores is ::  58.333333329404766  %
The Avet]rage Generation Score ::  68.85943874713085  %
Context Precision Score ::  78.57142856357143  %
Context Recall Score ::  38.0952380952381  %
Answer Relevancy Score ::  80.81411558949979  %
Faithfulness Score ::  56.904761904761905  %


Character Split Scores Ragas

In [121]:
character_df = pd.read_csv("data/evaluation/character_evaluration_test_ragas_001_raptor.csv", index_col=0)
character_df

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,Under what circumstances and to what extent th...,The Sellers are responsible for a breach of re...,['Liability. The Sellers’ Representative shal...,Except in the case of fraud the Sellers have n...,1.0,0.0,0.833333,0.949794
1,Would the Sellers be responsible if after the ...,"No, the Sellers would not be responsible if in...",['Liability. The Sellers’ Representative shal...,No.,1.0,1.0,0.800000,0.960716
2,How much is the escrow amount?,The escrow amount is not explicitly stated in ...,"['Escrow.\n\nAt Closing, Buyer will deposit th...","The escrow amount is equal to $1,000,000.",1.0,0.0,0.833333,0.909939
3,Is escrow amount greater than the Retention Am...,I don't have enough information to determine i...,['Escrow.\n\nPurchase Price Adjustment. If an...,No.,1.0,0.0,0.000000,0.000000
4,What is the purpose of the escrow?,The purpose of the escrow is to hold the Escro...,"['Escrow.\n\nAt Closing, Buyer will deposit th...",To serve as a recourse of the Buyer in case of...,1.0,1.0,0.800000,1.000000
5,May the Escrow Amount serve as a recourse for ...,"No, the recovery from the Escrow Amount is the...",['Recovery from the Escrow Amount shall consti...,No.,1.0,0.0,0.333333,0.851533
6,Are there any conditions to the closing?,The conditions to the closing include the prep...,['No Acquired Company will be required to incl...,No as the signing and closing are simultaneous.,1.0,0.0,1.000000,0.973219
7,Are Change of Control Payments considered a Se...,Change of Control Payments are not considered ...,['“Change of Control Payment” means (a) any bo...,Yes. (See defining of Sellers Transaction Expe...,1.0,1.0,1.000000,0.980234
8,Would the aggregate amount payable by the Buye...,"Yes, if the actual Closing Debt Amount is grea...",['Purchase Price Adjustment. If any of the Cl...,Yes. (See Section 2.07).,1.0,1.0,1.000000,0.935133
9,Does the Buyer need to pay the Employees Closi...,"Yes, at the Closing, the Buyer shall deposit t...",['Payments of Employee Closing Bonus. Prior to...,No. (See Section 2.10).,1.0,0.0,1.000000,0.902296


In [122]:
# calculate the average score of context precision per total score

context_score = character_df['context_precision'].sum() / character_df['context_precision'].count() * 100
context_score = character_df['context_recall'].sum() / character_df['context_recall'].count() * 100
faithfulness_score = character_df['faithfulness'].sum() / character_df['faithfulness'].count() * 100
answer_relevancy_score = character_df['answer_relevancy'].sum() / character_df['answer_relevancy'].count() * 100

avg_total_score = (context_precision_score + context_recall_score + faithfulness_score + answer_relevancy_score) / 4

avg_context_score = (context_precision_score + context_recall_score) / 2

avg_generate_score = (faithfulness_score + answer_relevancy_score) / 2

print("Average of All Scores is :: ", avg_total_score, " %")
print("Average of Context Scores is :: ", avg_context_score, " %")
print("The Avet]rage Generation Score :: ", avg_generate_score, " %")
print("Context Precision Score :: ", context_precision_score, " %")
print("Context Recall Score :: ", context_recall_score, " %")
print("Answer Relevancy Score :: ", answer_relevancy_score, " %")
print("Faithfulness Score :: ", faithfulness_score, " %")

Average of All Scores is ::  70.88593111492878  %
Average of Context Scores is ::  58.333333329404766  %
The Avet]rage Generation Score ::  83.4385289004528  %
Context Precision Score ::  78.57142856357143  %
Context Recall Score ::  38.0952380952381  %
Answer Relevancy Score ::  87.59134351519131  %
Faithfulness Score ::  79.28571428571428  %
